# 实例-搭建疫情数据分析网站

## 0 实例介绍

## 1 获取数据

## 2 网站`html`文件编辑

## 3 基础图像的绘制

本项目中，我们仍然使用`covid19`数据集，该数据集收集了31个身份53天内的疫情数据，我们参考JHU疫情分析以及百度大数据疫情分析网站，来制作基础的图像。

In [74]:
import pandas as pd 
import numpy as np 
from bokeh.plotting import figure, output_file, show
from bokeh.plotting import ColumnDataSource as CDS 
from bokeh.models import Toggle, Circle, CheckboxButtonGroup, RadioButtonGroup, Select, RadioGroup, HoverTool
from bokeh.layouts import layout, column, row
from bokeh.embed import file_html
from bokeh.events import ButtonClick
from bokeh.resources import CDN
from bokeh.io import save

In [6]:
covid19 = pd.read_csv('covid19.csv')
covid19['Observation Date'] = pd.to_datetime(covid19['Observation Date'], format = '%d-%m-%Y')

In [7]:
provincedata = covid19.groupby('Province/State')[['Confirmed', 'Observation Date']]

In [8]:
cityname = []
confirmed = []
deaths = []
latitude = []
longitude = []
date = list(provincedata)[0][1]['Observation Date']

for city in list(provincedata):
    cityname.append(city[0])
    confirmed.append(list(city[1]['Confirmed']))
    deaths.append(list(city[1]['Deaths']))
    latitude.append(city[1]['Latitude'].iloc[0])
    longitude.append(city[1]['Longitude'].iloc[0])

Confirmed = pd.DataFrame(pd.DataFrame(confirmed).values.T, index = date, columns = cityname)
Deaths = pd.DataFrame(pd.DataFrame(deaths).values.T, index = date, columns = cityname)

In [77]:
source = CDS(Confirmed)


TOOLTIPS = [
    ('date', '$x'),
    ('Confirmed Infections', '$y{0}'),
    ('Province', '@Province')
]

HoverTool.formatters = {'$x': 'datetime'}

pa = figure(plot_width = 800, plot_height = 400, x_axis_type = 'datetime', tooltips = TOOLTIPS)
pa.title.text = 'Confirmed Infections(Except Hubei)'
pa.title.text_color = 'navy'
pa.title.text_font_size = '20px'

hbconfirmed = pd.DataFrame(Confirmed['Hubei'])
hbconfirmed['date'] = hbconfirmed.index
Confirmed2 = Confirmed.drop('Hubei', axis = 1)

toggles = []
radios = []
finalshow = []
for data in range(30):
    confirmed = pd.DataFrame(Confirmed2.iloc[:, data])
    confirmed['Province'] = np.repeat(confirmed.columns[0], 54)
    confirmed['date'] = confirmed.index
    source = CDS(confirmed)
    p1 = pa.line(y = confirmed.columns[0] , x = 'date',source = source,  line_width = 2)
    finalshow.append(p1)
    p2 = pa.circle(y = confirmed.columns[0] , x = 'date',source = source, width = 0.6, color = 'navy', alpha = 0.5)

    selected_circle = Circle(fill_alpha = 1, fill_color = 'firebrick')
    unselected_circle = Circle(fill_alpha = 0.2, fill_color = 'blue')

    p2.selection_glyph = selected_circle
    p2.nonselection_glyph = unselected_circle


    #p1.selection_glyph = selected_line
    #p1.nonselection_glyph = unselected_line
    toggle1 = Toggle(label = confirmed.columns[0], button_type = 'default', active = True, width_policy = 'min', background = 'grey')
    toggle1.js_link('active', p1, 'visible')
    toggle1.js_link('active', p2, 'visible')
    toggles.append(toggle1)

pb = figure(plot_width = 800, plot_height = 400, x_axis_type = 'datetime', tooltips = TOOLTIPS)

pb.title.text = 'Hubei Confirmed Infections'
pb.title.text_color = 'navy'
pb.title.text_font_size = '20px'


source = CDS(hbconfirmed)
pb.line(y = 'Hubei', x = 'date',source = source, line_width = 2)
pb.circle(y = 'Hubei', x = 'date',source = source, width = 0.6, color = 'navy', alpha = 0.5)

toggles1 = toggles[0:10]
toggles2 = toggles[10:20]
toggles3 = toggles[20:]

layouts = layout([pa, toggles1,toggles2,toggles3,pb])

html = file_html(layouts, CDN, "myplot")

save(layouts)

'f:\\yihan\\Github存储库\\bokeh-intro\\myplot.html'